# エントロピー最大化と探索の話

**参考**

* [Fast Rates for Maximum Entropy Exploration](https://arxiv.org/abs/2303.08059)

強化学習ではたまに探索を促進するためにエントロピー正則化を導入する話があります。エントロピーってほんとに探索に効くんでしょうか？
エントロピー正則化と探索の理論を見てみましょう。

**表記**

* 今回は無報酬強化学習を考えます
* $\{p_h\}_h$：遷移確率
* $s_1$：初期状態
* 履歴：$I_h = \left(s_1, a_1, u_1, \ldots, s_{h-1}, a_{h-1}, u_{h-1}\right)$
    * $u_h$は$[0, 1]$の範囲の一様分布からの独立なサンプル。つまり、適当なノイズ。
* 行動：$a_h=\psi_h\left(I_h, s_h, u_h\right)$
* 履歴から行動へのマップ関数：$\psi_h:(\mathcal{S} \times \mathcal{A} \times [0,1])^{h-1} \times \mathcal{S} \times[0,1] \rightarrow \mathcal{A}$
* 方策：$\pi = \left(\psi_h\right)_{h \in[H]}$
    * マルコフ方策の場合は$a_h=\psi_h\left(s_h, u_h\right) \sim \pi_h\left(s_h\right)$
* 状態行動訪問分布：$d_h^\pi(s, a)$
* 訪問多面体：全ての訪問分布は次の多面体に属します
$$
\mathcal{K}_p \triangleq\left\{d=\left(d_h\right)_{h \in[H]}: \sum_{a \in \mathcal{A}} d_1(s, a)=\mathbb{1}\left\{s=s_1\right\} \forall s \in \mathcal{S},
\quad \sum_{a \in \mathcal{A}} d_{h+1}(s, a)=\sum_{\left(s^{\prime}, a^{\prime}\right) \in \mathcal{S} \times \mathcal{A}} p_h\left(s \mid s^{\prime}, a^{\prime}\right) d_h\left(s^{\prime}, a^{\prime}\right) \forall s \in \mathcal{S}, \quad \forall h \geq 1\right\}
$$
* 状態遷移の制約を外した訪問多面体：
$$
\mathcal{K} \triangleq\left\{d=\left(d_h\right)_{h \in[H]}: d_h(s, a) \geq 0, \forall(h, s, a) \in[H] \times \mathcal{S} \times \mathcal{A},
\quad \sum_{(s, a) \in \mathcal{S} \times \mathcal{A}} d_h(s, a)=1,\quad \forall h \in[H]\right\} .
$$
* 軌跡の集合：$\mathcal{T} \triangleq(\mathcal{S} \times \mathcal{A})^H=\left\{\left(s_1, a_1, \ldots, s_H, a_H\right):\left(s_h, a_h\right) \in \mathcal{S} \times \mathcal{A}, \forall h \in[H]\right\}$
    * 軌跡$m=\left(s_1, a_1, \ldots, s_H, a_H\right)$を方策$\pi$で生成する確率：$q^\pi(m) \triangleq \pi\left(a_1 \mid s_1\right) \prod_{h=2}^H p_{h-1}\left(s_h \mid s_{h-1}, a_{h-1}\right) \pi_h\left(a_h \mid s_h\right)$
    * この表記に乗っ取ると、訪問分布は $d_h^\pi(s, a)=\mathbb{E}_{\left(s_1, a_1, \ldots, s_H, a_H\right) \sim q^\pi}\left[\mathbb{1}\left\{(s, a)=\left(s_h, a_h\right)\right\}\right]$の形で書けることに注意。
* $(s, a)$が$h$ステップで$t$エピソード目までに訪問された回数：$n_h^t(s, a) \triangleq \sum_{i=1}^t \mathbb{1}\left\{\left(s_h^i, a_h^i\right)=(s, a)\right\}$
* $n_h^t(s^\prime \mid s, a) \triangleq \sum_{i=1}^t \mathbb{1}\left\{\left(s_h^i, a_h^i, s_{h+1}^i\right)=\left(s, a, s^{\prime}\right)\right\}$
* 経験分布：$\widehat{p}_h^t\left(s^{\prime} \mid s, a\right)=n_h^t\left(s^{\prime} \mid s, a\right) / n_h^t(s, a)$

以降ではエントロピーを最大化するRLを考えるわけですが、問題設定の中では様々な分布が出てきます。
つまり、最大化するエントロピーの選択肢が色々あるわけですね。いくつか見てみましょう。

## 訪問エントロピーの最大化

まず、次で定義される**訪問エントロピー**を考えてみましょう。

$$
\mathcal{H}_{\mathrm{visit}}\left(d^\pi\right) \triangleq \sum_{h=1}^H \mathcal{H}\left(d_h^\pi\right)
$$

これを最大化する方策を$\pi^{\star, \mathrm{VE}} \in \arg \max _\pi \mathcal{H}_{\text {visit }}\left(d^\pi\right)$とします。
つまり、$\pi^{\star, \mathrm{VE}}$はいろんな状態行動を訪問するわけですね。

---

**訪問エントロピー最大化探索（MVEE）**

MVEEでは、次のように無報酬MDPを探索します。
1. エピソード$t$の最初に、エージェントはエピソード$t-1$までの履歴から方策$\pi^t$を決定します。
2. $\pi_t$に沿って新しい履歴を収集します
3. 停止時刻$\tau$に従ってデータの収集を止めるか判断します。
4. ジェネラルな方策$\hat{\pi}$を出力します。

つまり、MVEEは$\left(\left(\pi^t\right)_{t \in \mathbb{N}}, \tau, \widehat{\pi}\right)$で表記されます。
また、次を満たすアルゴリズム$\left(\left(\pi^t\right)_{t \in \mathbb{N}}, \tau, \widehat{\pi}\right)$をMVEEにおける$(\varepsilon, \delta)$-PACアルゴリズムとよびます。

$$
\mathbb{P}\left(\mathcal{H}_{\text {visit }}\left(d^{\pi^{\star, \mathrm{VE}}}\right)-\mathcal{H}_{\text {visit }}\left(d^{\widehat{\pi}}\right) \leq \varepsilon\right) \geq 1-\delta
$$

実はこのMVEE問題は、次の予測ゲームを解くのと等価になります。

$$
\begin{aligned}
\max _{d \in \mathcal{K}_p} \mathcal{H}_{\mathrm{visit}}(d) & =\max _{d \in \mathcal{K}_p} \min _{\bar{d} \in \mathcal{K}} \sum_{(h, s, a)} d_h(s, a) \log \frac{1}{\bar{d}_h(s, a)} \\
& =\min _{\bar{d} \in \mathcal{K}} \max _{d \in \mathcal{K}_p} \sum_{(h, s, a)} d_h(s, a) \log \frac{1}{\bar{d}_h(s, a)}
\end{aligned}
$$

この2行目はSionのミニマックス定理ってやつですね（TODO:ゲーム理論）。
1行目は

$$
\mathcal{H}_{\mathrm{visit}}(d)=\sum_{h=1}^H \mathcal{H}\left(d_h\right)=\sum_{h=1}^H \mathcal{H}\left(d_h\right)+\mathrm{KL}\left(d_h, d_h\right)=\min _{\bar{d} \in \mathcal{K}} \sum_{h=1}^H\left(\mathcal{H}\left(d_h\right)+\mathrm{KL}\left(d_h, \bar{d}_h\right)\right)=\min _{\bar{d} \in \mathcal{K}} \sum_{(h, s, a)} d_h(s, a) \log \frac{1}{\bar{d}_h(s, a)}
$$

によって簡単に導出されます。

直感的な説明：
* Minプレーヤー側：$\mathrm{KL}\left(d_h, \bar{d}_h\right)$を最小にしようとします。つまり、Maxプレーヤー側の分布を予測しようとします。
* Maxプレーヤー側：Minプレーヤーの予測を崩そうとがんばります。

これを実現するアルゴリズムを見てみましょう。

### MVEEのためのEntGameアルゴリズム　

EntGameアルゴリズムは上記の予測ゲームを実装したものです。

**予測プレーヤー**

Mixture-Forecasterってやつで実現します（[Prediction, Learning, and Games](https://ii.uni.wroc.pl/~lukstafi/pmwiki/uploads/AGT/Prediction_Learning_and_Games.pdf)の９章参照。）
予測プレーヤーは次のように分布を推定します：
* $\bar{d}^t_h(s, a) = \bar{n}_h^{t-1}(s, a) /\left(t+t_0\right)$ 
  * $t_0=S A n_0$とします。ここで、$n_0$は事前カウントです。
  * $\bar{n}_h^t(s, a)=n_h^t(s, a)+n_0$
  * $n_h^t(s, a)$はサンプラープレーヤーが$(s, a)$を訪れた回数です。

**サンプラープレーヤー**

楽観的なbest-responseで実現します。
$$\begin{aligned} \bar{Q}_h^t(s, a) & =\log \frac{1}{\bar{d}_h^{t+1}(s, a)}+\widehat{p}_h^t \bar{V}_{h+1}^t(s, a)+b_h^t(s, a) \\ \bar{V}_h^t(s) & =\operatorname{clip}\left(\max _{a \in \mathcal{A}} \bar{Q}_h^t(s, a), 0, \log \left(t / n_0+S A\right) H\right)\end{aligned}$$

を解いて、その貪欲方策$\pi_h^{t+1}(s) \in \arg \max _{\pi \in \Delta_A} \pi \bar{Q}_h^t(s)$ で実現します。
ここで、

$$
b_h^t(s, a)=\sqrt{\frac{2 H^2 \log ^2(t+S A) \cdot \alpha^{\mathrm{KL}}\left(\delta, n_h^t(s, a)\right)}{n_h^t(s, a)}}
$$

です。疑似アルゴリズムはめんどくさいので論文に任せます。
この予測プレーヤーとサンプラープレーヤーを$T$回繰り返すことを考えます。
このとき、$T=\widetilde{\mathcal{O}}\left(\frac{H^4 S^2 A}{\varepsilon^2}+\frac{H S A}{\varepsilon}\right)$でEntGameが$(\varepsilon, \delta)$-PACであることが証明できます。

TODO: 証明

## 軌跡エントロピーの最大化（MTEE）

先程は訪問分布のエントロピーを最大化しました。今度は軌跡の分布のエントロピーを最大化してみます。つまり、次の**軌跡エントロピー**を考えます：

$$
\mathcal{H}_{\text {traj }}\left(q^\pi\right) \triangleq \mathcal{H}\left(q^\pi\right)=\sum_{m \in \mathcal{T}} q^\pi(m) \log \frac{1}{q^\pi(m)}
$$

$\pi^{\star, \mathrm{TE}} \in \arg \max _\pi \mathcal{H}_{\text {traj }}\left(q^\pi\right)$を軌跡エントロピーを最大化する方策とします。
MVEEの話と同じようにして、アルゴリズム$\left(\left(\pi^t\right)_{t \in \mathbb{N}}, \tau, \widehat{\pi}\right)$は次を満たすときMTEEについて$(\varepsilon, \delta)$-PACと言います。

$$
\mathbb{P}\left(\mathcal{H}_{\text {traj }}\left(q^{\pi^{\star, \mathrm{TE}}}\right)-\mathcal{H}_{\text {traj }}\left(q^{\widehat{\pi}}\right) \leq \varepsilon\right) \leq 1-\delta
$$

